# Testing framework
Though ollama has also a python library to access the models, the local resources of my computer are limited so we are going to use instead a cloud provider to access different LLMs. The idea is to send requests to the API with the questions of the dataset we prepared. The answers will be then collected and evaluated later.

Environment:
* hyperstack.cloud 

Models:
* Llama-2-7b-chat-hf
* mistralai_Mistral-7B-Instruct-v0.2

Dataset:
* questions_df (with sports and science questions from polymarket)

# Load dataset

In [1]:
import pandas as pd
import os
cwd = os.getcwd()
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
data_path = os.path.join(parent_dir, "data")
questions_df = pd.read_pickle(os.path.join(data_path,"questions_df"))
questions_df.head()

C:\Users\ARosa\AppData\Local\Temp\ipykernel_21704\3061992813.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,question,answer,category,year
0,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023
1,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023
2,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023
3,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023
4,Will Netherlands qualify for the World Cup in ...,Yes,sport,2023


# Using the oobabooga/text-generationwebui


In [2]:
import requests
host = 'https://var-meal-providers-influence.trycloudflare.com:443'
url = host+"/v1/chat/completions"
print(url)

https://var-meal-providers-influence.trycloudflare.com:443/v1/chat/completions


In [3]:
headers = {
    "Content-Type": "application/json"
}
def prepare_questions(year: int = None):
    '''
    This function will prepare the list with the binary questions from the questions_df dataset
    :param year: optional filter to pass only questions from a specific year
    :return: a list with all the questions
    '''
    if year is not None:
        return questions_df[questions_df["year"]==year].question.values
    return questions_df.question.values

def get_response(question: str):
    message = [{"role": "user", "content": question}]
    data = {
        "mode": "chat",
        "messages": message
    }

    response = requests.post(url, headers=headers, json=data, verify=False)
    return response.json()['choices'][0]['message']['content']

## Mistral responses

In [6]:
mistral_responses = []
questions = prepare_questions()
for question in questions:
    print(f"question = {question}")
    question_with_prob = f'''Answer the following question by giving as output a json response with two fields: 
the probability of the answer and the confidence interval of the answer. The question is: {question}'''
    assistant_message = get_response(question_with_prob)
    print(f"response={assistant_message}")
    mistral_responses.append(assistant_message)

question = In the cricket IPL game, scheduled for May 14th in 2023, which team is going to win: Rajasthan or Bangalore?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'var-meal-providers-influence.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response=I'm an AI, I don't have the ability to predict or know the outcome of a cricket match before it happens. However, I can give you some general information or historical data that might help you make an informed decision. For instance, if we look at the most recent performance and head-to-head records, we find that:

1. Rajasthan won 5 of the last 10 matches against Bangalore.
2. Bangalore won 5 of the last 10 matches against Rajasthan.

So, based on this data, we could estimate that the probability of Rajasthan winning is around 0.52 (52%) and the probability of Bangalore winning is around 0.48 (48%). However, these probabilities only reflect historic performance and do not account for any potential injuries, team composition changes, or unexpected factors that could influence the outcome of the match.

As for the confidence interval, it's important to keep in mind that predicting sports outcomes is inherently uncertain. Based on the binary nature of the question ( Rajasthan or

c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'var-meal-providers-influence.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
  "probability": 0.45,
  "confidence_interval": [0.4, 0.5]
}
question = In the cricket IPL game, scheduled for May 14th in 2023, which team is going to win: Rajasthan or Bangalore?


...

response={
  "probability": 0.2,
  "confidence_interval": [0.1, 0.4]
}
The AI's response indicates that it believes there is a 20% chance that the room-temperature superconductor discovered in 2023 is real. The confidence interval suggests that the true probability could be as low as 10% or as high as 40%.

question = Iceland volcanic eruption by November 17th 2023?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'var-meal-providers-influence.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
  "probability": 0.05,
  "confidence_interval": {
   "lower": 0.01,
   "upper": 0.08
  }
}
I'm sorry for the low probability and wide confidence interval. I can only base my answer on current geological data and historical patterns, and there's no definitive evidence of an imminent volcanic eruption in Iceland by November 17th, 2023. However, it's important to note that geological activity can be unpredictable and Iceland is known for its volcanic activity.
question = Is the Earth flat?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'var-meal-providers-influence.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
"probability": 0.0,
"confidence_interval": {
"lower_bound": 0.0,
"upper_bound": 0.0
}
}


In [7]:
questions_df["mistral_responses_iter1"]= mistral_responses

In [9]:
# new model and new host
host = "https://solomon-namespace-hu-killing.trycloudflare.com"
url = host+"/v1/chat/completions"

llama2_responses = []
for question in questions:
    print(f"question = {question}")
    question_with_prob = f'''Answer the following question by giving as output a json response with two fields: 
the probability of the answer and the confidence interval of the answer. The question is: {question}'''
    assistant_message = get_response(question_with_prob)
    print(f"response={assistant_message}")
    llama2_responses.append(assistant_message)

question = In the cricket IPL game, scheduled for May 14th in 2023, which team is going to win: Rajasthan or Bangalore?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'solomon-namespace-hu-killing.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
"probs": 0.7,
"confidence_interval": [0.6, 0.85]
}
question = In the NBA basketball game, scheduled for October 27th in 2023, which team is going to win: Pistons or Hornets?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'solomon-namespace-hu-killing.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
"probability": 0.7,
"confidenceInterval": [0.5,0.9]
}

question = In the cricket IPL game, scheduled for May 14th in 2023, which team is going to win: Rajasthan or Bangalore?


...

response={
"probability": 0.6,
"confidence_interval": [0.4, 0.8]
}

question = Iceland volcanic eruption by November 17th 2023?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'solomon-namespace-hu-killing.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
"probability": 0.02,
"confidence interval": (0.01, 0.05)
}

question = Is the Earth flat?


c:\Users\ARosa\Documents\valory_llm\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'solomon-namespace-hu-killing.trycloudflare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


response={
"probability": 0.000000456,
"confidenceInterval": [0.000000241, 0.000000718]
}



In [10]:
questions_df["llama2_responses_iter1"] = llama2_responses

In [11]:
questions_df.head()

,question,answer,category,year,mistral_responses_iter1,llama2_responses_iter1
0,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023,"I'm an AI, I don't have the ability to predict...","{\n""probs"": 0.7,\n""confidence_interval"": [0.6,..."
1,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023,"{\n ""probability"": 0.45,\n ""confidence_inter...","{\n""probability"": 0.7,\n""confidenceInterval"": ..."
2,"In the cricket IPL game, scheduled for May 14t...",Bangalore,sport,2023,"{\n ""probability"": 0.51,\n ""confidence_inter...","{\n""probability"": 0.7,\n""confidence_interval"":..."
3,"In the NBA basketball game, scheduled for Octo...",Pistons,sport,2023,"{\n ""probability"": 0.5,\n ""confidence_interv...","{\n""probability"": 0.75,\n""confidence"": 0.9\n}\n"
4,Will Netherlands qualify for the World Cup in ...,Yes,sport,2023,"{\n""probability"": 0.8,\n""confidence_interval"":...","{\n""proba"": 0.75,\n""conf"": [0.6, 0.9]\n}"


In [12]:
questions_df.to_pickle(os.path.join(data_path,'responses_df'))

# References

https://github.com/oobabooga/text-generation-webui/wiki/12-%E2%80%90-OpenAI-API#examples
https://cheatsheet.md/llm-leaderboard/ollama.en
